    Imports

In [53]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

    Main page

In [21]:
url = 'https://www.vogue.com/fashion-shows/latest-shows'
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Contenido HTML de la página web
    html_content = response.text
    print(f"Pagina obtenida. \nhead de la página: \n{html_content[:30]}")
else:
    print("Error al obtener la página:", response.status_code)
    
# Crear un objeto BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

Pagina obtenida. 
head de la página: 
<!DOCTYPE html><html lang="en-


    Encontrar pagina de los diseñadores (parte dict y imagenes WIP)

In [24]:
# Encontrar todos los elementos <a> en la página
enlaces = soup.find_all('a')
year = "2024"
designer_dict = {}
# df = pd.DataFrame(columns=['designer', 'img'])

# Imprimir los enlaces encontrados
for enlace in enlaces:
    # print(f"Descargando enlace: {enlace.get('href')}")
    enlace = enlace.get('href')
    if year in enlace:
        designer_page = "https://www.vogue.com"+ enlace
        designer_name = enlace.split('/')[-1]
        print(f"Descargando enlace de {designer_name}: {designer_page}")
        response = requests.get(designer_page)
        if response.status_code == 200:
            html_content = response.text
        #     soup = BeautifulSoup(html_content, 'html.parser')
            # print(soup.find_all('img'))
            for img in soup.find_all('img'):
                if "SummaryItemSimpleHed" not in img.get('alt'):
                    continue
                # print(img.get('src'))
                if img.get('src') is not None and img.get('src').endswith('.jpg'):
                    if designer_name not in designer_dict:
                        designer_dict[designer_name] = [img.get('src')]
                    else:
                        designer_dict[designer_name].append(img.get('src'))
        else:
            print("Error al obtener la página:", response.status_code)

Descargando enlace de sunflower: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/sunflower
Descargando enlace de soulland: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/soulland
Descargando enlace de helmstedt: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/helmstedt
Descargando enlace de caro-editions: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/caro-editions
Descargando enlace de mark-kenly-domino-tan: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/mark-kenly-domino-tan
Descargando enlace de paolina-russo: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/paolina-russo
Descargando enlace de baum-und-pferdgarten: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/baum-und-pferdgarten
Descargando enlace de henrik-vibskov: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/henrik-vibskov
Descargando enlace de stine-goya: https://www.vogue.com/fashion-shows/copenhagen-fall-2024/stine-goya
Descargando enlace de mfpen:

    DF con diferente tamaño vertical (teniendo en cuenta el key con mas len()del  dict y los demás a None)

In [84]:
max_len = max(len(lst) for lst in designer_dict.values())
for key in designer_dict:
    len_diff = max_len - len(designer_dict[key])
    if len_diff > 0:
        designer_dict[key] += [None] * len_diff

df = pd.DataFrame(designer_dict)

    Encontrar página de all pictures en cada diseñador

In [49]:
designer_page = "https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia"

response = requests.get(designer_page)
if response.status_code == 200:
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    # sacar la pagina de ver las imagenes
    boton = soup.find_all(class_="RunwayShowPageGalleryCta-fmTQJF FtdcL")
    # sacar el link de la pagina de imagenes
    link = boton[0].find('a').get('href')


In [50]:
link

'/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection'

    Encontrar imagenes en allpics (ESTOY ATASCADO) // en esta parte siempre recoge la misma imagen aunque se itere sobre varias páginas

In [54]:
collection_page = "https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection"
# range de 10 como ejemplo
for i in range(10):
    collection_page_iter = f"{collection_page}#{str(i+1)}"
    print(collection_page_iter, "\n")
    response = requests.get(f"{collection_page}#{str(i+1)}")
    if response.status_code == 200:
        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')
        for img in soup.find_all(class_="ResponsiveAssetWrapper-gdcDmm eeYyjw"):
            img = img.find('img')
            src = img.get('src')
            if src is not None and src.endswith('.jpg'):
                print(src)
        time.sleep(2)  # Esperar un segundo antes de la siguiente solicitud

    else:
        print("Error al obtener la página:", response.status_code)



https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection#1 

https://assets.vogue.com/photos/65ba546ae1fe3c1de6ebc725/master/w_2560%2Cc_limit/00001-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection#2 

https://assets.vogue.com/photos/65ba546ae1fe3c1de6ebc725/master/w_2560%2Cc_limit/00001-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection#3 

https://assets.vogue.com/photos/65ba546ae1fe3c1de6ebc725/master/w_2560%2Cc_limit/00001-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://www.vogue.com/fashion-shows/fall-2024-ready-to-wear/sara-battaglia/slideshow/collection#4 

https://assets.vogue.com/photos/65ba546ae1fe3c1de6ebc725/master/w_2560%2Cc_limit/00001-sara-battaglia-fall-2024-ready-to-wear-credit-brand.jpg
https://www.vogue.com/fashio